In [1]:
DBPATH = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/test_DB/"
DBNAME = "test.db"
DBRESDIR = "RMSResources/"
import sys
import logging
logger = logging.getLogger("rmspool")
logger.setLevel("INFO")
logger.addHandler(logging.StreamHandler(sys.stdout))
from rmsp import rmscore
rms = rmscore.ResourceManagementSystem(f"{DBPATH}{DBNAME}", f"{DBPATH}{DBRESDIR}")
from rmsp import rmsbuilder
rmspool = rmsbuilder.RMSProcessWrapPool(rms, 8)
rmsb = rmsbuilder.RMSUnrunTasksBuilder(rmspool)
from rmsp import rmsutils
rms.set_scriptID("503ed0ede5424701a68c6c562eb671f4")

Checking tasks to skip...
Checking pending tasks to submit...
Total pending pids available to submit: 0 / 0
Checking tasks to skip...
Skipped 46 tasks that were done before.
Checking tasks to skip...
Checking pending tasks to submit...
Total pending pids available to submit: 0 / 0


In [2]:
exec(rmsutils.convert_import_str('''
from commandtemplate.conda import run_template_bash
''',
rmsvar = "rmsb"
))
conda_env = "DI"

In [3]:
PROJECT_DIR_d = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/"
PROJECT_DIR_r = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/"

# Merged data

## Merge replicates

In [4]:
# Merge C1a and C1b

prefixes = ["C1a", "C1b"]
output_prefix = "C1"
f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"samtools merge -o {o} " + " ".join([f"<(samtools view -b -h {{i{idx}}} chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 chrX)" for idx in range(len(prefixes))]),
	conda_env = conda_env, 
	output_func = output_func,
	**{f"i{idx}": rmsb.file_from_path(f"{PROJECT_DIR_d}RawAlignments/{prefix}_dedup.bam") for idx, prefix in enumerate(prefixes)},
	o = f"{PROJECT_DIR_d}Analysis/{output_prefix}_dedup_chr1-22-X.bam"
)
f = lambda **kwargs: [f"{kwargs['i']}.bai"]
output_func = rms.register_pipe(f)
run_template_bash(
	"samtools index {i}", 
	conda_env = conda_env,
	output_func = output_func,
	i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/{output_prefix}_dedup_chr1-22-X.bam")
)

## Get bigwig files (full read length)

In [5]:
# Extract R2

f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"samtools view -b -f 128 {i} > {o}",
	conda_env = conda_env,
	output_func = output_func,
	i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X.bam"),
	o = f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2.bam",
)
f = lambda **kwargs: [f"{kwargs['i']}.bai"]
output_func = rms.register_pipe(f)
run_template_bash(
	"samtools index {i}", 
	conda_env = conda_env,
	output_func = output_func,
	i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2.bam"),
)

In [6]:
# Convert bam to bigwig

f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
run_template_bash(
	"bamCoverage -b {i} -o {o} --filterRNAstrand {strand} --binSize 1 --numberOfProcessors {thread}",
	conda_env = conda_env,
	output_func = output_func,
	i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2.bam"),
	bami = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2.bam.bai"),
	o = f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2_coverage_pl.bw",
	strand = "forward",
	thread = 32
)
run_template_bash(
	"bamCoverage -b {i} -o {o} --filterRNAstrand {strand} --binSize 1 --numberOfProcessors {thread}",
	conda_env = conda_env,
	output_func = output_func,
	i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2.bam"),
	bami = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2.bam.bai"),
	o = f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X_R2_coverage_mn.bw",
	strand = "reverse",
	thread = 32
)

# Subsampling

## Create subsampled bam files

In [4]:
# Check original seq depth

original = f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X.bam"
! samtools flagstat $original

88200416 + 0 in total (QC-passed reads + QC-failed reads)
88200416 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
88200416 + 0 mapped (100.00% : N/A)
88200416 + 0 primary mapped (100.00% : N/A)
88200416 + 0 paired in sequencing
44100208 + 0 read1
44100208 + 0 read2
88200416 + 0 properly paired (100.00% : N/A)
88200416 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [7]:
subsample_dict = {
	"1M": 1000000,
	"5M": 5000000,
	"10M": 10000000,
	"20M": 20000000,
	"30M": 30000000
	}

In [8]:
f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
for subsample_name, n in subsample_dict.items():
	run_template_bash(
		"biodatatools subsample_bam -i {i} -o {o} -n {n} -seed {seed}",
		conda_env = conda_env,
		output_func = output_func,
		i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_dedup_chr1-22-X.bam"),
		o = f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}.bam",
		n = n,
		seed = n
	)
f = lambda **kwargs: [f"{kwargs['i']}.bai"]
output_func = rms.register_pipe(f)
for subsample_name, n in subsample_dict.items():
	run_template_bash(
		"samtools index {i}", 
		conda_env = conda_env,
		output_func = output_func,
		i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}.bam")
	)

In [5]:
# Check seq depth after downsampling

downsampled = f"{PROJECT_DIR_d}Analysis/C1_10M.bam"
! samtools flagstat $downsampled

20000000 + 0 in total (QC-passed reads + QC-failed reads)
20000000 + 0 primary
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
20000000 + 0 mapped (100.00% : N/A)
20000000 + 0 primary mapped (100.00% : N/A)
20000000 + 0 paired in sequencing
10000000 + 0 read1
10000000 + 0 read2
20000000 + 0 properly paired (100.00% : N/A)
20000000 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


## Peak calling

In [9]:
# Convert bam to bigwig (5' only)

chrom_size_file = f"{PROJECT_DIR_r}genomes/human/hg38.chrom.sizes.filtered"
for subsample_name, n in subsample_dict.items():
	f = lambda **kwargs: [kwargs['o'] + "_5pl.bw", kwargs['o'] + "_5mn.bw", kwargs['o'] + "_3pl.bw", kwargs['o'] + "_3mn.bw"]
	output_func = rms.register_pipe(f)
	run_template_bash(
		"biodatatools process_PROcap_bam_to_bigwig -i {i} -g {g} -o {o} -paired_end True -rna_strand {rna_strand}",
		conda_env = conda_env,
		output_func = output_func,
		i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}.bam"),
		g = rms.file_from_path(chrom_size_file),
		o = f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}",
		rna_strand = "reverse"
	)

In [10]:
# Peak calling using PINTS

f = lambda **kwargs: [f"{kwargs['output_dir']}/{kwargs['file_prefix']}_1_bidirectional_peaks.bed", f"{kwargs['output_dir']}/{kwargs['file_prefix']}_1_divergent_peaks.bed", f"{kwargs['output_dir']}/{kwargs['file_prefix']}_1_unidirectional_peaks.bed"]
output_func = rms.register_pipe(f)
for subsample_name, n in subsample_dict.items():
	bwpl = f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_5pl.bw"
	bwmn = f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_5mn.bw"
	prefix = f"C1_{subsample_name}"
	run_template_bash(
		"pints_caller --save-to {output_dir} --file-prefix {file_prefix} --bw-pl {bwpl} --bw-mn {bwmn} --exp-type {exp_type} --min-lengths-opposite-peaks 5 --thread {thread}", 
		conda_env = conda_env, 
		output_func = output_func,
		output_dir = f"{PROJECT_DIR_d}Analysis/",
		file_prefix = prefix, 
		bwpl = rmsb.file_from_path(bwpl),
		bwmn = rmsb.file_from_path(bwmn),
		exp_type = "PROcap",
		thread = 32
	)

## Get bigwig files (full read length)

In [11]:
f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
for subsample_name, n in subsample_dict.items():
	run_template_bash(
		"samtools view -b -f 128 {i} > {o}",
		conda_env = conda_env,
		output_func = output_func,
		i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}.bam"),
		o = f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2.bam",
	)
f = lambda **kwargs: [f"{kwargs['i']}.bai"]
output_func = rms.register_pipe(f)
for subsample_name, n in subsample_dict.items():
	run_template_bash(
		"samtools index {i}", 
		conda_env = conda_env,
		output_func = output_func,
		i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2.bam"),
	)

In [12]:
f = lambda **kwargs: [kwargs['o']]
output_func = rms.register_pipe(f)
for subsample_name, n in subsample_dict.items():
	run_template_bash(
		"bamCoverage -b {i} -o {o} --filterRNAstrand {strand} --binSize 1 --numberOfProcessors {thread}",
		conda_env = conda_env,
		output_func = output_func,
		i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2.bam"),
		bami = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2.bam.bai"),
		o = f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2_coverage_pl.bw",
		strand = "forward",
		thread = 32
	)
	run_template_bash(
		"bamCoverage -b {i} -o {o} --filterRNAstrand {strand} --binSize 1 --numberOfProcessors {thread}",
		conda_env = conda_env,
		output_func = output_func,
		i = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2.bam"),
		bami = rmsb.file_from_path(f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2.bam.bai"),
		o = f"{PROJECT_DIR_d}Analysis/C1_{subsample_name}_R2_coverage_mn.bw",
		strand = "reverse",
		thread = 32
	)

In [13]:
u = rmsb.execute_builder()

In [14]:
rmspool.print_stat()

COMPLETE: 0
RUNNING: 0
ERROR: 0
PENDING_READY: 0
PENDING_NOT_READY: 0
